In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda
import time

print("⚠️  CUDA C++ is the PRIMARY learning material!")

---

## Part 1: cuBLAS Overview

### What is cuBLAS?

```
cuBLAS = CUDA Basic Linear Algebra Subroutines

Features:
- GPU-accelerated BLAS (levels 1, 2, 3)
- Highly optimized for NVIDIA hardware
- Tensor Core acceleration (GEMM)
- Near-peak performance

BLAS Levels:
┌─────────┬──────────────────────────┬─────────────┐
│ Level   │ Operations               │ Complexity  │
├─────────┼──────────────────────────┼─────────────┤
│ Level 1 │ Vector-Vector (dot, axpy)│ O(n)        │
│ Level 2 │ Matrix-Vector (gemv)     │ O(n²)       │
│ Level 3 │ Matrix-Matrix (gemm)     │ O(n³)       │
└─────────┴──────────────────────────┴─────────────┘
```

### BLAS Naming Convention

```
Example: cublasSgemm, cublasDgemm

Pattern: cublas[Precision][Operation]

Precision codes:
  S = float (single precision)
  D = double
  C = complex float
  Z = complex double
  H = half precision (FP16)

Common operations:
  gemm = GEneral Matrix Multiply
  gemv = GEneral Matrix Vector
  axpy = A*X Plus Y
  dot  = DOT product
```

### 🔷 CUDA C++ Implementation (Primary)

### 🔶 Python/CuPy (Optional - Quick Testing)

In [ ]:
%%writefile cublas_gemm.cu
// cublas_gemm.cu - Matrix multiplication with cuBLAS
#include <stdio.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

int main() {
    int M = 1024, N = 1024, K = 1024;
    
    size_t size_A = M * K * sizeof(float);
    size_t size_B = K * N * sizeof(float);
    size_t size_C = M * N * sizeof(float);
    
    // Host memory
    float *h_A = (float*)malloc(size_A);
    float *h_B = (float*)malloc(size_B);
    float *h_C = (float*)malloc(size_C);
    
    // Initialize
    for (int i = 0; i < M * K; i++) h_A[i] = rand() / (float)RAND_MAX;
    for (int i = 0; i < K * N; i++) h_B[i] = rand() / (float)RAND_MAX;
    
    // Device memory
    float *d_A, *d_B, *d_C;
    cudaMalloc(&d_A, size_A);
    cudaMalloc(&d_B, size_B);
    cudaMalloc(&d_C, size_C);
    
    cudaMemcpy(d_A, h_A, size_A, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, h_B, size_B, cudaMemcpyHostToDevice);
    
    // Create cuBLAS handle
    cublasHandle_t handle;
    cublasCreate(&handle);
    
    // GEMM: C = alpha * A * B + beta * C
    float alpha = 1.0f;
    float beta = 0.0f;
    
    // Note: cuBLAS uses column-major order!
    // For row-major C = A * B, we compute C^T = B^T * A^T
    // cublasSgemm(handle, transB, transA, N, M, K, ...)
    
    // Warmup
    cublasSgemm(handle,
                CUBLAS_OP_N, CUBLAS_OP_N,  // No transpose
                N, M, K,                    // Dimensions
                &alpha,
                d_B, N,                     // B and leading dimension
                d_A, K,                     // A and leading dimension
                &beta,
                d_C, N);                    // C and leading dimension
    cudaDeviceSynchronize();
    
    // Benchmark
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    for (int i = 0; i < 100; i++) {
        cublasSgemm(handle,
                    CUBLAS_OP_N, CUBLAS_OP_N,
                    N, M, K,
                    &alpha, d_B, N, d_A, K,
                    &beta, d_C, N);
    }
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    ms /= 100;
    
    double flops = 2.0 * M * N * K;
    double tflops = flops / (ms * 1e9);
    
    printf("cuBLAS SGEMM: %d×%d×%d\n", M, K, N);
    printf("Time: %.3f ms\n", ms);
    printf("Performance: %.2f TFLOPS\n", tflops);
    
    // Cleanup
    cublasDestroy(handle);
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);
    
    return 0;
}

// Compile: nvcc cublas_gemm.cu -o cublas_gemm -lcublas

In [ ]:
!nvcc -arch=sm_75 -o cublas_gemm cublas_gemm.cu -lcublas
!./cublas_gemm

In [ ]:
# CuPy provides easy access to cuBLAS
try:
    import cupy as cp
    CUPY_AVAILABLE = True
except ImportError:
    print("CuPy not available - install with: pip install cupy-cuda12x")
    CUPY_AVAILABLE = False

In [ ]:
if CUPY_AVAILABLE:
    # CuPy uses cuBLAS under the hood
    M, K, N = 1024, 1024, 1024
    
    A = cp.random.rand(M, K).astype(cp.float32)
    B = cp.random.rand(K, N).astype(cp.float32)
    
    # Warmup
    C = A @ B
    cp.cuda.Stream.null.synchronize()
    
    # Benchmark
    start = cp.cuda.Event()
    end = cp.cuda.Event()
    
    start.record()
    for _ in range(100):
        C = A @ B
    end.record()
    end.synchronize()
    
    ms = cp.cuda.get_elapsed_time(start, end) / 100
    flops = 2 * M * N * K
    tflops = flops / (ms * 1e9)
    
    print(f"CuPy (cuBLAS) GEMM: {M}×{K}×{N}")
    print(f"Time: {ms:.3f} ms")
    print(f"Performance: {tflops:.2f} TFLOPS")

---

## Part 2: Row-Major vs Column-Major

### The Column-Major Challenge

In [ ]:
def explain_column_major():
    """Explain cuBLAS column-major layout."""
    print("cuBLAS Column-Major Layout")
    print("=" * 50)
    print()
    print("Row-major (C/C++, NumPy default):")
    print("  A = [[1, 2, 3],    Memory: [1,2,3,4,5,6]")
    print("       [4, 5, 6]]")
    print()
    print("Column-major (Fortran, cuBLAS):")
    print("  A = [[1, 2, 3],    Memory: [1,4,2,5,3,6]")
    print("       [4, 5, 6]]")
    print()
    print("Solutions for row-major data:")
    print("  1. Transpose before/after (memory overhead)")
    print("  2. Use CUBLAS_OP_T to transpose on-the-fly")
    print("  3. Compute C = B^T × A^T (equals (A×B)^T)")
    print()
    print("Trick: For row-major C = A × B:")
    print("  Call gemm(B, A) instead of gemm(A, B)")
    print("  Because in column-major: C^T = B^T × A^T")

explain_column_major()

---

## Part 3: cuBLAS API Deep Dive

### GEMM Parameters

```cpp
cublasStatus_t cublasSgemm(
    cublasHandle_t handle,      // cuBLAS context
    cublasOperation_t transa,   // CUBLAS_OP_N, CUBLAS_OP_T, CUBLAS_OP_C
    cublasOperation_t transb,
    int m,                      // Rows of op(A) and C
    int n,                      // Columns of op(B) and C
    int k,                      // Columns of op(A), rows of op(B)
    const float *alpha,         // Scalar α
    const float *A, int lda,    // A and its leading dimension
    const float *B, int ldb,    // B and its leading dimension
    const float *beta,          // Scalar β
    float *C, int ldc           // C and its leading dimension
);

// Computes: C = α × op(A) × op(B) + β × C
```

### Common cuBLAS Functions

```cpp
// Level 1: Vector operations
cublasSaxpy(handle, n, &alpha, x, 1, y, 1);  // y = α*x + y
cublasSdot(handle, n, x, 1, y, 1, &result);  // result = x·y
cublasSnrm2(handle, n, x, 1, &result);       // result = ||x||₂

// Level 2: Matrix-vector
cublasSgemv(handle, trans, m, n, &alpha, A, lda, x, 1, &beta, y, 1);

// Level 3: Matrix-matrix
cublasSgemm(handle, transA, transB, m, n, k, ...);  // C = A*B
cublasSsyrk(handle, uplo, trans, n, k, ...);        // C = A*A^T
cublasStrsm(handle, side, uplo, trans, diag, ...);  // Triangular solve
```

In [ ]:
def cublas_quick_reference():
    """Quick reference for cuBLAS operations."""
    print("cuBLAS Quick Reference")
    print("=" * 60)
    print()
    print("LEVEL 1 (vector-vector):")
    print("  Saxpy:  y = α*x + y              O(n)")
    print("  Sdot:   s = x^T * y              O(n)")
    print("  Snrm2:  s = ||x||_2              O(n)")
    print("  Sscal:  x = α*x                  O(n)")
    print()
    print("LEVEL 2 (matrix-vector):")
    print("  Sgemv:  y = α*A*x + β*y          O(n²)")
    print("  Ssymv:  y = α*A*x + β*y (symm)   O(n²)")
    print("  Strsv:  x = A^{-1}*b (triangular)O(n²)")
    print()
    print("LEVEL 3 (matrix-matrix):")
    print("  Sgemm:  C = α*A*B + β*C          O(n³)")
    print("  Ssyrk:  C = α*A*A^T + β*C        O(n³)")
    print("  Strsm:  B = α*A^{-1}*B           O(n³)")
    print()
    print("Prefixes: S=float, D=double, C=complex, Z=double-complex")

cublas_quick_reference()

---

## Part 4: Performance Comparison

In [ ]:
# Our tiled kernel for comparison
TILE_SIZE = 16

@cuda.jit
def matmul_tiled(A, B, C, M, N, K):
    As = cuda.shared.array((TILE_SIZE, TILE_SIZE), dtype=np.float32)
    Bs = cuda.shared.array((TILE_SIZE, TILE_SIZE), dtype=np.float32)
    
    tx, ty = cuda.threadIdx.x, cuda.threadIdx.y
    row = cuda.blockIdx.y * TILE_SIZE + ty
    col = cuda.blockIdx.x * TILE_SIZE + tx
    
    total = 0.0
    
    for t in range((K + TILE_SIZE - 1) // TILE_SIZE):
        a_col = t * TILE_SIZE + tx
        if row < M and a_col < K:
            As[ty, tx] = A[row, a_col]
        else:
            As[ty, tx] = 0.0
        
        b_row = t * TILE_SIZE + ty
        if b_row < K and col < N:
            Bs[ty, tx] = B[b_row, col]
        else:
            Bs[ty, tx] = 0.0
        
        cuda.syncthreads()
        
        for k in range(TILE_SIZE):
            total += As[ty, k] * Bs[k, tx]
        
        cuda.syncthreads()
    
    if row < M and col < N:
        C[row, col] = total

In [ ]:
def compare_performance(sizes=[256, 512, 1024, 2048]):
    """Compare our kernel vs cuBLAS (via CuPy)."""
    if not CUPY_AVAILABLE:
        print("CuPy not available for cuBLAS comparison")
        return
    
    print(f"{'Size':<12} {'Tiled (ms)':<12} {'Tiled GFLOPS':<14} {'cuBLAS (ms)':<12} {'cuBLAS GFLOPS':<14} {'Ratio':<8}")
    print("=" * 85)
    
    for size in sizes:
        M = K = N = size
        flops = 2 * M * N * K
        
        # Numba tiled kernel
        A_np = np.random.rand(M, K).astype(np.float32)
        B_np = np.random.rand(K, N).astype(np.float32)
        C_np = np.zeros((M, N), dtype=np.float32)
        
        d_A = cuda.to_device(A_np)
        d_B = cuda.to_device(B_np)
        d_C = cuda.to_device(C_np)
        
        grid = ((N + TILE_SIZE - 1) // TILE_SIZE, (M + TILE_SIZE - 1) // TILE_SIZE)
        block = (TILE_SIZE, TILE_SIZE)
        
        # Warmup
        matmul_tiled[grid, block](d_A, d_B, d_C, M, N, K)
        cuda.synchronize()
        
        iterations = 20
        start = time.perf_counter()
        for _ in range(iterations):
            matmul_tiled[grid, block](d_A, d_B, d_C, M, N, K)
        cuda.synchronize()
        tiled_ms = (time.perf_counter() - start) / iterations * 1000
        tiled_gflops = flops / (tiled_ms * 1e6)
        
        # CuPy (cuBLAS)
        A_cp = cp.asarray(A_np)
        B_cp = cp.asarray(B_np)
        
        # Warmup
        C_cp = A_cp @ B_cp
        cp.cuda.Stream.null.synchronize()
        
        start_ev = cp.cuda.Event()
        end_ev = cp.cuda.Event()
        
        start_ev.record()
        for _ in range(iterations):
            C_cp = A_cp @ B_cp
        end_ev.record()
        end_ev.synchronize()
        
        cublas_ms = cp.cuda.get_elapsed_time(start_ev, end_ev) / iterations
        cublas_gflops = flops / (cublas_ms * 1e6)
        
        ratio = cublas_gflops / tiled_gflops
        
        print(f"{size}×{size}×{size:<4} {tiled_ms:<12.3f} {tiled_gflops:<14.1f} {cublas_ms:<12.3f} {cublas_gflops:<14.1f} {ratio:.1f}x")

try:
    compare_performance()
except Exception as e:
    print(f"Comparison failed: {e}")

---

## Part 5: cuBLAS Best Practices

In [ ]:
def cublas_best_practices():
    """Best practices for cuBLAS usage."""
    print("cuBLAS Best Practices")
    print("=" * 60)
    print()
    print("1. HANDLE MANAGEMENT")
    print("   ✓ Create handle once, reuse for all operations")
    print("   ✗ Don't create/destroy handle per call")
    print()
    print("2. MEMORY LAYOUT")
    print("   cuBLAS uses column-major (Fortran) order")
    print("   For row-major (C/C++): C = A*B → call gemm(B,A)")
    print()
    print("3. MEMORY ALIGNMENT")
    print("   cudaMalloc provides aligned memory (good)")
    print("   For submatrices, ensure lda is multiple of 32")
    print()
    print("4. BATCH OPERATIONS")
    print("   Use cublasSgemmBatched for many small GEMMs")
    print("   Use cublasSgemmStridedBatched for uniform strides")
    print()
    print("5. TENSOR CORES (Ampere+)")
    print("   Use cublasGemmEx with CUBLAS_COMPUTE_32F_FAST_TF32")
    print("   Or half precision for maximum Tensor Core usage")
    print()
    print("6. ERROR CHECKING")
    print("   Always check cublasStatus_t return values")
    print("   Use cublasGetStatusString for error messages")

cublas_best_practices()

---

## Part 6: Complete cuBLAS Example

### CUDA C++ Production Code

In [ ]:
%%writefile cublas_complete.cu
// cublas_complete.cu - Production-ready cuBLAS usage
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <cublas_v2.h>

#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        fprintf(stderr, "CUDA error at %s:%d: %s\n", __FILE__, __LINE__, \
                cudaGetErrorString(err)); \
        exit(1); \
    } \
}

#define CHECK_CUBLAS(call) { \
    cublasStatus_t stat = call; \
    if (stat != CUBLAS_STATUS_SUCCESS) { \
        fprintf(stderr, "cuBLAS error at %s:%d: %d\n", __FILE__, __LINE__, stat); \
        exit(1); \
    } \
}

class CuBLASGemm {
private:
    cublasHandle_t handle;
    
public:
    CuBLASGemm() {
        CHECK_CUBLAS(cublasCreate(&handle));
    }
    
    ~CuBLASGemm() {
        cublasDestroy(handle);
    }
    
    // Row-major: C = alpha * A * B + beta * C
    void gemm(int M, int N, int K,
              float alpha, const float* A, const float* B,
              float beta, float* C) {
        // For row-major, swap A and B and swap dimensions
        CHECK_CUBLAS(cublasSgemm(handle,
                                  CUBLAS_OP_N, CUBLAS_OP_N,
                                  N, M, K,
                                  &alpha,
                                  B, N,
                                  A, K,
                                  &beta,
                                  C, N));
    }
};

int main() {
    CuBLASGemm gemm;
    
    int M = 1024, K = 512, N = 2048;
    
    // Allocate and initialize host memory
    size_t size_A = M * K * sizeof(float);
    size_t size_B = K * N * sizeof(float);
    size_t size_C = M * N * sizeof(float);
    
    float *h_A = (float*)malloc(size_A);
    float *h_B = (float*)malloc(size_B);
    float *h_C = (float*)malloc(size_C);
    
    for (int i = 0; i < M * K; i++) h_A[i] = rand() / (float)RAND_MAX;
    for (int i = 0; i < K * N; i++) h_B[i] = rand() / (float)RAND_MAX;
    
    float *d_A, *d_B, *d_C;
    CHECK_CUDA(cudaMalloc(&d_A, size_A));
    CHECK_CUDA(cudaMalloc(&d_B, size_B));
    CHECK_CUDA(cudaMalloc(&d_C, size_C));
    
    CHECK_CUDA(cudaMemcpy(d_A, h_A, size_A, cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_B, h_B, size_B, cudaMemcpyHostToDevice));
    
    // Perform GEMM
    gemm.gemm(M, N, K, 1.0f, d_A, d_B, 0.0f, d_C);
    CHECK_CUDA(cudaDeviceSynchronize());
    
    printf("cuBLAS Complete Example: %d×%d @ %d×%d = %d×%d\n", M, K, K, N, M, N);
    printf("GEMM completed successfully!\n");
    
    CHECK_CUDA(cudaFree(d_A));
    CHECK_CUDA(cudaFree(d_B));
    CHECK_CUDA(cudaFree(d_C));
    free(h_A); free(h_B); free(h_C);
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o cublas_complete cublas_complete.cu -lcublas
!./cublas_complete

---

## Exercises

### Exercise 1: Batched GEMM

In [ ]:
# TODO: Use CuPy to perform batched matrix multiplication
# Compare performance: loop vs batched

def batched_gemm_comparison():
    """Compare loop GEMM vs batched GEMM."""
    if not CUPY_AVAILABLE:
        print("CuPy required for this exercise")
        return
    
    batch_size = 64
    M = K = N = 128
    
    # Create batched matrices
    A = cp.random.rand(batch_size, M, K).astype(cp.float32)
    B = cp.random.rand(batch_size, K, N).astype(cp.float32)
    
    # Loop approach
    C_loop = [A[i] @ B[i] for i in range(batch_size)]
    
    # Batched approach (CuPy uses batched GEMM automatically)
    C_batch = A @ B  # This uses batched cuBLAS under the hood
    
    print(f"Batch size: {batch_size}, Matrix size: {M}×{K}×{N}")
    # TODO: Add timing comparison

batched_gemm_comparison()

### Exercise 2: Mixed Precision GEMM

In [ ]:
# TODO: Compare float32 vs float16 GEMM performance
# Note: float16 uses Tensor Cores on supported hardware

def mixed_precision_comparison():
    if not CUPY_AVAILABLE:
        print("CuPy required")
        return
    
    M = K = N = 2048
    
    # Float32
    A32 = cp.random.rand(M, K).astype(cp.float32)
    B32 = cp.random.rand(K, N).astype(cp.float32)
    
    # Float16
    A16 = A32.astype(cp.float16)
    B16 = B32.astype(cp.float16)
    
    # TODO: Benchmark and compare
    print("Mixed precision GEMM comparison")

mixed_precision_comparison()

---

## Summary

### When to Use cuBLAS

| Scenario | Recommendation |
|----------|----------------|
| Standard BLAS operations | Always use cuBLAS |
| Custom fused kernels | Write custom CUDA |
| Batch small matrices | cublasBatched variants |
| Maximum throughput | Mixed precision + Tensor Cores |

### Key Takeaways

1. **cuBLAS is highly optimized** - usually 5-10x faster than custom kernels
2. **Column-major layout** - understand the memory layout conversion
3. **Handle management** - create once, reuse many times
4. **Tensor Cores** - use half precision or TF32 for maximum performance

### CUDA C++ Pattern

```cpp
cublasHandle_t handle;
cublasCreate(&handle);

// Row-major C = A × B
cublasSgemm(handle,
            CUBLAS_OP_N, CUBLAS_OP_N,
            N, M, K,         // Note: N, M, K order
            &alpha,
            B, N,            // B first, ldb=N
            A, K,            // A second, lda=K
            &beta,
            C, N);           // C, ldc=N

cublasDestroy(handle);
```

### Week 6 Complete!
Next week: Memory Optimization Deep Dive (occupancy, caching, unified memory)